In [109]:
import pandas as pd
import codecs
import chardet
import numpy as np

Считываем файлы

In [35]:
first = pd.read_csv("2.tmp",encoding='utf-8', sep='\n', header=None)
second = pd.read_csv("3.tmp", encoding='utf-8', sep='\n', header=None)
third = pd.read_csv("4.tmp", encoding='utf-8', sep='\n', header=None)

Переименовываем колонки

In [36]:
first.columns = ['line']

Добавляем колонку target - номер файла

In [37]:
first['target'] = [1]*len(first)

Отчищаем от значений NaN и выводим на экран

In [38]:
first.dropna(inplace=True)
first.head()

,line,target
0,Bag End先生的Bilbo Baggins先生宣布，他不久將以特別的輝煌的聚會慶祝他的第...,1
3,什麼，人們普遍相信，Bag End山充滿了寶藏的隧道。如果這還不夠名聲，那麼他也有長期的活力...,1
6,青年以及（著名地）無窮無盡的財富似乎是不公平的。他們說：“這將需要支付。” “這不是自然的，...,1
8,仍然與親屬（除了當然是薩克維爾 - 巴金斯）的訪問條件，他在貧窮和不重要的家庭的愛好中有許多...,1
30,＃＃”你說得對，爸爸！“Gaffer說。 “不是巴剋土地的白蘭地酒在老森林裡生活，但他們似乎...,1


То же самое проделываем с остальными файлами

In [39]:
second.columns = ['line']
second['target'] = [2]*len(second)
second.dropna(inplace=True)
second.head()

,line,target
0,When Mr. Bilbo Baggins of Bag End announced th...,2
5,"Bilbo was very rich and very peculiar, and had b",2
23,‘It will have to be paid f,2
26,But so far trouble had not come;,2
35,The eldest o,2


In [40]:
third.columns = ['line']
third['target'] = [3]*len(third)
third.dropna(inplace=True)
third.head()

,line,target
0,Bag EndのBilbo Baggins氏は、18歳の誕生日を特別な壮大なパーティでまもな...,3
2,失踪と予期せぬ復帰以来、60年にわたりシャイアの驚異であった。彼が旅行から持ち帰った富は今や...,3
4,に信じられていました。それが名声のために十分でないならば、彼の長年の活力も驚くべきものでした...,3
5,かかりましたが、バギンズ氏にはほとんど影響がないようでした。 90歳で彼は50歳とほとんど同...,3
8,"富を所有すべきであるということは不公平に思えました。 ""それは支払われなければならないだろう...",3


Соединяем файлы

In [111]:
data = pd.concat([first, second, third], ignore_index=True)
data.head()

,line,target
0,Bag End先生的Bilbo Baggins先生宣布，他不久將以特別的輝煌的聚會慶祝他的第...,1
1,什麼，人們普遍相信，Bag End山充滿了寶藏的隧道。如果這還不夠名聲，那麼他也有長期的活力...,1
2,青年以及（著名地）無窮無盡的財富似乎是不公平的。他們說：“這將需要支付。” “這不是自然的，...,1
3,仍然與親屬（除了當然是薩克維爾 - 巴金斯）的訪問條件，他在貧窮和不重要的家庭的愛好中有許多...,1
4,＃＃”你說得對，爸爸！“Gaffer說。 “不是巴剋土地的白蘭地酒在老森林裡生活，但他們似乎...,1


In [113]:
data.index = range(len(data))
data.head()

,line,target
0,Bag End先生的Bilbo Baggins先生宣布，他不久將以特別的輝煌的聚會慶祝他的第...,1
1,什麼，人們普遍相信，Bag End山充滿了寶藏的隧道。如果這還不夠名聲，那麼他也有長期的活力...,1
2,青年以及（著名地）無窮無盡的財富似乎是不公平的。他們說：“這將需要支付。” “這不是自然的，...,1
3,仍然與親屬（除了當然是薩克維爾 - 巴金斯）的訪問條件，他在貧窮和不重要的家庭的愛好中有許多...,1
4,＃＃”你說得對，爸爸！“Gaffer說。 “不是巴剋土地的白蘭地酒在老森林裡生活，但他們似乎...,1


In [114]:
size = len(data)
print(len(data))
data = data[:size]

1943


Поробуем предсказывать метки классов, используя логистическую регрессию, в качестве признаков будем использовать n-граммы

In [115]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

In [122]:
vectorizer = CountVectorizer(ngram_range=(3,5), analyzer='char_wb', lowercase=False, max_df=0.84, binary=True)

In [123]:
%%time
data_matrix = vectorizer.fit_transform(data['line'])

Wall time: 638 ms


In [124]:
data_matrix

<1943x133091 sparse matrix of type '<class 'numpy.int64'>'
	with 205262 stored elements in Compressed Sparse Row format>

Оцениваем результат, используя KFold и accuracy_score

In [125]:
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score

In [126]:
%%time
score = []
predictions = []
# Используйте процедуру KFold для проверки качества классификаторов, не забываем перемешивать данные
kf = KFold(len(data), n_folds=10, shuffle=True)
for train_index, test_index in kf:
    X_train, X_test = data_matrix[train_index], data_matrix[test_index]
    y_train, y_test = data['target'][train_index], data['target'][test_index]
    
    model = LogisticRegression(penalty='l1', n_jobs=-1, random_state=45)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    score.append(accuracy_score(y_test, predictions))

Wall time: 4.28 s


Посмотрим на результат классификации

In [130]:
print("Accurasy scores: {}\n\n".format(score))
print("Mean accurasy score: {}".format(np.array(score).mean()))

Accurasy scores: [0.93333333333333335, 0.96923076923076923, 0.94871794871794868, 0.96391752577319589, 0.93814432989690721, 0.96907216494845361, 0.95360824742268047, 0.96907216494845361, 0.98453608247422686, 0.96907216494845361]


Mean accurasy score: 0.9598704731694422


Точность достаточно высокая

In [25]:
# type(first['line'][0])
line = first['line'][0].split(" ")
line

['Bag End先生的Bilbo Baggins先生宣布，他不久將以特別的輝煌的聚會慶祝他的第二十一歲']

In [28]:
# for ch in second['line'][0]:
#     print(chardet.detect(codecs.encode(ch)), sep=" ")

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
{'encoding': 'ascii', 'confiden

In [26]:
chardet.charsetprober.CharSetProber.(codecs.encode("first['line'][0]"))

{'confidence': 1.0, 'encoding': 'ascii', 'language': ''}